In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from importlib import reload
import os
from os.path import join
import subprocess
import sys
sys.path.append('../../') # lets us import ptype package from the subdir
#import ptype.
import time
from collections.abc import Sequence
from joblib import Parallel, delayed

from xhistogram.xarray import histogram

import soundings.utils as utils
import soundings.mr_analysis as mra
from soundings.mr_analysis import SoundingQuery

In [4]:
ds = xr.open_dataset("/glade/work/dkimpara/ptype-aggs/gfs_new_york_1.nc", engine='netcdf4')
ds 

<xarray.Dataset>
Dimensions:            (predtype: 8, case_study_day: 3, init_hr: 4, step: 18,
                        heightAboveGround: 21, bin: 999)
Coordinates:
  * predtype           (predtype) object 'ML_cicep' 'cicep' ... 'crain'
  * case_study_day     (case_study_day) object 'new_york_1-20221215' ... 'new...
  * init_hr            (init_hr) int64 0 6 12 18
  * step               (step) timedelta64[ns] 01:00:00 02:00:00 ... 18:00:00
  * heightAboveGround  (heightAboveGround) int64 0 250 500 ... 4500 4750 5000
  * bin                (bin) float64 -59.95 -59.85 -59.75 ... 39.65 39.75 39.85
Data variables:
    num_obs            (predtype, case_study_day, step, init_hr) float64 ...
    t_h_fabv           (predtype, case_study_day, step, init_hr) float64 ...
    dpt_h_fabv         (predtype, case_study_day, step, init_hr) float64 ...
    wb_h_fabv          (predtype, case_study_day, step, init_hr) float64 ...
    t_h_mean           (predtype, case_study_day, step, init_hr, heightAboveGround) float32 ...
    dpt_h_mean         (predtype, case_study_day, step, init_hr, heightAboveGround) float32 ...
    wb_h_mean          (predtype, case_study_day, step, init_hr, heightAboveGround) float32 ...
    t_h_hist           (predtype, case_study_day, step, init_hr, heightAboveGround, bin) float64 ...
    dpt_h_hist         (predtype, case_study_day, step, init_hr, heightAboveGround, bin) float64 ...
    wb_h_hist          (predtype, case_study_day, step, init_hr, heightAboveGround, bin) float64 ...
    valid_time         (init_hr, case_study_day, step) datetime64[ns] ...

In [5]:
utils.count_nulls(ds)

num_obs: 0
t_h_fabv: 321
dpt_h_fabv: 321
wb_h_fabv: 321
t_h_mean: 6741
dpt_h_mean: 6741
wb_h_mean: 6741
t_h_hist: 6734259
dpt_h_hist: 6738255
wb_h_hist: 6734259
valid_time: 0


In [6]:
nullt = ds.t_h_fabv.where(ds.t_h_fabv.isnull(), drop=True)

In [8]:
nullt

<xarray.DataArray 't_h_fabv' (predtype: 1, case_study_day: 1, step: 1,
                              init_hr: 1)>
array([[[[nan]]]])
Coordinates:
  * predtype        (predtype) object 'cicep'
  * case_study_day  (case_study_day) object 'kentucky-20220223'
  * init_hr         (init_hr) int64 22
  * step            (step) timedelta64[ns] 14:00:00

In [13]:
ds.t_h_mean.where(ds.t_h_mean.isnull(), drop=True).step

<xarray.DataArray 'step' (step: 1)>
array([50400000000000], dtype='timedelta64[ns]')
Coordinates:
  * step     (step) timedelta64[ns] 14:00:00
Attributes:
    long_name:      time since forecast_reference_time
    standard_name:  forecast_period

In [10]:
ds.t_h_hist.where(ds.t_h_hist.isnull(), drop=True)

<xarray.DataArray 't_h_hist' (predtype: 1, case_study_day: 1, step: 1,
                              init_hr: 1, heightAboveGround: 21, bin: 999)>
array([[[[[[nan, nan, nan, ..., nan, nan, nan],
           [nan, nan, nan, ..., nan, nan, nan],
           [nan, nan, nan, ..., nan, nan, nan],
           ...,
           [nan, nan, nan, ..., nan, nan, nan],
           [nan, nan, nan, ..., nan, nan, nan],
           [nan, nan, nan, ..., nan, nan, nan]]]]]])
Coordinates:
  * predtype           (predtype) object 'cicep'
  * case_study_day     (case_study_day) object 'kentucky-20220223'
  * init_hr            (init_hr) int64 22
  * step               (step) timedelta64[ns] 14:00:00
  * heightAboveGround  (heightAboveGround) int64 0 250 500 ... 4500 4750 5000
  * bin                (bin) float64 -59.95 -59.85 -59.75 ... 39.65 39.75 39.85

In [74]:
query = SoundingQuery([ds])

In [17]:
rap = utils.open_ds_dkimpara('2200','rap', 'c', 'step')

In [20]:
rapsel = rap.sel(step=[np.timedelta64(50400000000000, 'ns')])

In [55]:
def get_coords_of_value(ds):
    for model in ['', 'ML_']:
        for ptype in ['cicep', 'crain', 'csnow', 'cfrzr']:
            ptype = model + ptype
            masked_ds = ds.where(rapsel[ptype] == 1)
            mins = icep.crain.argmin(dim=('y', 'x'))
            print(f"{ptype}: {mins['x'].values} {mins['y'].values}")
            mins = icep.crain.argmax(dim=('y', 'x'))
            print(f"{ptype}: {mins['x'].values} {mins['y'].values}")

In [56]:
get_coords_of_value(rapsel)

cicep: [[450]] [[262]]
cicep: [[450]] [[262]]
crain: [[450]] [[262]]
crain: [[450]] [[262]]
csnow: [[450]] [[262]]
csnow: [[450]] [[262]]
cfrzr: [[450]] [[262]]
cfrzr: [[450]] [[262]]
ML_cicep: [[450]] [[262]]
ML_cicep: [[450]] [[262]]
ML_crain: [[450]] [[262]]
ML_crain: [[450]] [[262]]
ML_csnow: [[450]] [[262]]
ML_csnow: [[450]] [[262]]
ML_cfrzr: [[450]] [[262]]
ML_cfrzr: [[450]] [[262]]
